In [3]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from flask_cors import CORS
from rembg import remove
from PIL import Image
import io
import time

app = Flask(__name__)
CORS(app)

os.chdir("C:\\Users\\TisGJRRR\\EcoSort_Revision")
model_path = "Models/EcoSortV3.h5"
model = tf.keras.models.load_model(model_path)

UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

class_labels = ['cardboard', 'glass', 'metal', 'paper', 'plastic']

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def generate_unique_filename(original_filename):
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    ext = original_filename.rsplit('.', 1)[1].lower()
    return f"image_{timestamp}.{ext}"

def remove_background(image_path):
    """Remove the background using rembg."""
    try:
        with open(image_path, 'rb') as input_file:
            input_data = input_file.read()

        # Remove the background
        output_data = remove(input_data)

        # Convert the output to an image
        output_image = Image.open(io.BytesIO(output_data))
        output_image_path = image_path.rsplit('.', 1)[0] + '_no_bg.png'
        output_image.save(output_image_path)

        return output_image_path
    except Exception as e:
        print(f"Error in remove_background: {str(e)}")
        raise

def predict_trash_class(image_path):
    try:
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        predictions = model.predict(x)
        predicted_class_index = np.argmax(predictions)
        predicted_class = class_labels[predicted_class_index]
        certainty = float(predictions[0][predicted_class_index] * 100)

        return predicted_class, certainty
    except Exception as e:
        print(f"Error in predict_trash_class: {str(e)}")
        raise

def safe_file_removal(filepath):
    """Safely remove a file if it exists."""
    try:
        if os.path.exists(filepath):
            os.remove(filepath)
    except Exception as e:
        print(f"Error removing file {filepath}: {str(e)}")

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if not allowed_file(file.filename):
        return jsonify({'error': 'Invalid file type. Allowed types: png, jpg, jpeg'}), 400

    try:
        # Generate unique filename
        filename = generate_unique_filename(secure_filename(file.filename))
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        
        # Save uploaded file
        file.save(filepath)

        # Process image
        image_no_bg_path = remove_background(filepath)
        predicted_class, certainty = predict_trash_class(image_no_bg_path)

        # Clean up files
        safe_file_removal(filepath)
        safe_file_removal(image_no_bg_path)

        return jsonify({
            'class': predicted_class,
            'certainty': certainty
        }), 200

    except Exception as e:
        # Clean up files in case of error
        if 'filepath' in locals():
            safe_file_removal(filepath)
        if 'image_no_bg_path' in locals():
            safe_file_removal(image_no_bg_path)
        
        error_message = str(e)
        print(f"Error processing image: {error_message}")
        return jsonify({'error': f'Error processing image: {error_message}'}), 500

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [04/Jan/2025 01:26:28] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


...

INFO:werkzeug:127.0.0.1 - - [04/Jan/2025 01:27:23] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


INFO:werkzeug:127.0.0.1 - - [04/Jan/2025 01:27:49] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


INFO:werkzeug:127.0.0.1 - - [04/Jan/2025 01:27:59] "POST /predict HTTP/1.1" 200 -
